In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
#import lightgbm as lgb
#import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

In [ ]:
#Questions:
#What is XGBoost
#How to know outlier bounderies
#How to determine xgboost params
#What to do with DMatrix
#Error Reason: image not found for lightgbm, will solve later and skip lightGBM part
#Jupyter server down when training through xgboost
#Difference between np.random.seed() and random.seed()
#pd.merge

In [8]:
%pwd
%cd '/Users/xiaoyingliu/desktop/zillow'

/Users/xiaoyingliu/Desktop/zillow


In [4]:
#########XGBoost
prop=pd.read_csv('properties_2016.csv')


/Users/xiaoyingliu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
train=pd.read_csv('train_2016_v2.csv')

In [5]:
#process data for XGBoost
for i in prop.columns:
    prop[i]=prop[i].fillna(-1)
    if prop[i].dtype=='object':
        lbl=LabelEncoder()
        lbl.fit(list(prop[i].values))
        prop[i]=lbl.transform(list(prop[i].values))

In [8]:
train_df=train.merge(prop,how='left',on='parcelid')
x_train=train_df.drop(['parcelid','logerror','transactiondate'],axis=1)
x_test=prop.drop(['parcelid'],axis=1)

In [9]:
#shape
print('shape train: {}\nshape test: {}'. format(x_train.shape, x_test.shape))

shape train: (90275, 57)
shape test: (2985217, 57)


In [10]:
#drop out outliers  
#we can extract column in df python using df.ColumnName or df['ColumnName']
train_df=train_df[train_df.logerror>-0.4]
train_df=train_df[train_df.logerror<0.419]
y_train=train_df['logerror'].values.astype(np.float32)
y_mean=np.mean(y_train)

In [11]:
print('After removing outliers: ')
print('shape train: {}\nshape test: {}'. format(x_train.shape, x_test.shape))      

After removing outliers: 
shape train: (90275, 57)
shape test: (2985217, 57)


In [12]:
#Run XGBoost
#set xgboost params
xgb_params={
    'eta':0.037,
    'max_depth':5,
    'subsample':0.80,
    'objective':'reg:linear',
    'eval_metric' :'mae',
    'lambda':0.8,
    'alpha':0.4,
    'base_score':y_mean,
    'silent':1    
}

In [13]:
dtrain=xgb.DMatrix(x_train,y_train)
dtest=xgb.DMatrix(x_test)
num_boost_rounds=250

In [ ]:
#This chunk does not work since
#train_model
#model=xgb.train(dict(xgb_params,silent=1),dtrain,num_boost_round=num_boost_rounds)
#predicting
#xgb_pred1=model.predict(dtest)

In [1]:
#print predictions
#print(pd.DataFrame(xgb.pred1).head())

NameError: name 'pd' is not defined

In [ ]:
#Run xgboost again using different params
#Combine 2-time results weighted average

In [ ]:
#OLS

In [6]:
np.random.seed(17)
random.seed(17)

None


In [15]:
train=pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'])
properties=pd.read_csv('properties_2016.csv')
submission=pd.read_csv('sample_submission.csv')

In [16]:
print(len(train),len(properties),len(submission))

90275 2985217 2985217


In [18]:
#Get year,month, quarter as new features from df
def get_features(df):
    df['transactiondate']=pd.to_datetime(df['transactiondate'])
    df['transactiondate_year']=df['transactiondate'].dt.year
    df['transactiondate_month']=df['transactiondate'].dt.month
    df['transactiondate_quarter']=df['transactiondate'].dt.quarter
    df=df.fillna(-1.0)
    return df

In [19]:
def MAE(y,ypred):
    #logerror=log(Zestimate)-log(SalesPrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) /len(y)

In [21]:
train=get_features(train[col])

[1, 2, 3]